In [10]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import holidays
import joblib

# Cargar el DataFrame desde el archivo CSV
data_path = os.path.join(os.getcwd(), 'data')  # Ruta al directorio 'data'
train = pd.read_csv(os.path.join(data_path, 'test_store1.csv'))

# Asegurarse de que la columna 'date' esté en formato datetime
train['date'] = pd.to_datetime(train['date'], errors='coerce')

# Asegurarse de que solo hay un registro por fecha y artículo
train = train.groupby(['date', 'item'], as_index=False)['sales'].sum()

# Eliminar filas con fechas inválidas
train = train.dropna(subset=['date'])



# Extraer características de tiempo adicionales
mean_sales['dayofweek'] = mean_sales['date'].dt.dayofweek
mean_sales['month'] = mean_sales['date'].dt.month
mean_sales['year'] = mean_sales['date'].dt.year
mean_sales['dayofyear'] = mean_sales['date'].dt.dayofyear
mean_sales['is_weekend'] = mean_sales['date'].dt.dayofweek >= 5

# Agregar información sobre días festivos
us_holidays = holidays.MEX(years=[2013, 2014, 2015, 2016, 2017])
mean_sales['is_holiday'] = mean_sales['date'].isin(us_holidays)

# Convertir booleanos a enteros
mean_sales['is_weekend'] = mean_sales['is_weekend'].astype(int)
mean_sales['is_holiday'] = mean_sales['is_holiday'].astype(int)

# Dividir los datos en conjuntos de entrenamiento y prueba
X = mean_sales[['date', 'item', 'dayofweek', 'month', 'year', 'dayofyear', 'is_weekend', 'is_holiday']]
y = mean_sales['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear variables dummy para la característica 'item' (si hay más de un artículo)
X_train = pd.get_dummies(X_train, columns=['item'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['item'], drop_first=True)

# Calcular la fecha mínima de X_train para la codificación de fecha
date_min = X_train['date'].min()

# Codificar la característica de fecha como días desde una fecha de referencia
X_train['date'] = (X_train['date'] - date_min).dt.days
X_test['date'] = (X_test['date'] - date_min).dt.days

# Crear interacciones entre características
for col in X_train.columns:
    if col.startswith('item_'):
        X_train[f'date_{col}'] = X_train['date'] * X_train[col]
        X_test[f'date_{col}'] = X_test['date'] * X_test[col]

# Crear y entrenar el modelo de regresión Ridge con búsqueda de hiperparámetros
ridge = Ridge()
ridge_param_grid = {'alpha': [0.1, 1.0, 10.0, 100.0]}
ridge_grid_search = GridSearchCV(ridge, ridge_param_grid, cv=5, scoring='r2')
ridge_grid_search.fit(X_train, y_train)

# Mejor modelo Ridge encontrado
best_ridge_model = ridge_grid_search.best_estimator_

# Evaluar el modelo Ridge
y_ridge_pred = best_ridge_model.predict(X_test)
ridge_mse = mean_squared_error(y_test, y_ridge_pred)
ridge_r2 = r2_score(y_test, y_ridge_pred)
print(f"Ridge - MSE: {ridge_mse}")
print(f"Ridge - R2: {ridge_r2}")

# Guardar el modelo entrenado
joblib.dump(best_ridge_model, 'best_ridge_model.pkl')

# Guardar la fecha mínima utilizada para la codificación
joblib.dump(date_min, 'date_min.pkl')

# Guardar las columnas de X
joblib.dump(X_train.columns.tolist(), 'X_columns.pkl')


/tmp/ipykernel_12763/1856232561.py:34: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  mean_sales['is_holiday'] = mean_sales['date'].isin(us_holidays)


Ridge - MSE: 138.61335284723447
Ridge - R2: 0.757082986615664


['X_columns.pkl']

In [11]:
# Datos de prueba
test = [
  {'date': '2018-01-01', 'item': 1},
  {'date': '2024-01-01', 'item': 2},
  {'date': '2024-01-01', 'item': 3},
  {'date': '2024-01-01', 'item': 4},
  {'date': '2024-01-01', 'item': 5},
]

# Convertir a DataFrame
test_df = pd.DataFrame(test)

# Asegurarse de que la columna 'date' esté en formato datetime
test_df['date'] = pd.to_datetime(test_df['date'], errors='coerce')

# Extraer características de tiempo adicionales
test_df['dayofweek'] = test_df['date'].dt.dayofweek
test_df['month'] = test_df['date'].dt.month
test_df['year'] = test_df['date'].dt.year
test_df['dayofyear'] = test_df['date'].dt.dayofyear
test_df['is_weekend'] = test_df['date'].dt.dayofweek >= 5

# Agregar información sobre días festivos
us_holidays = holidays.US()
test_df['is_holiday'] = test_df['date'].isin(us_holidays)

# Convertir booleanos a enteros
test_df['is_weekend'] = test_df['is_weekend'].astype(int)
test_df['is_holiday'] = test_df['is_holiday'].astype(int)

# Crear variables dummy para la característica 'item'
test_df = pd.get_dummies(test_df, columns=['item'], drop_first=True)

# Codificar la característica de fecha como días desde una fecha de referencia
test_df['date'] = (test_df['date'] - date_min).dt.days

# Crear interacciones entre características
for col in test_df.columns:
    if col.startswith('item_'):
        test_df[f'date_{col}'] = test_df['date'] * test_df[col]

# Asegurarse de que las columnas en el conjunto de prueba coinciden con las del entrenamiento
expected_columns = set(X_train.columns)
current_columns = set(test_df.columns)
missing_columns = expected_columns - current_columns

for col in missing_columns:
    test_df[col] = 0

# Ordenar las columnas para que coincidan
test_df = test_df[X_train.columns]

# Escalar los datos de prueba
test_df_scaled = test_df

# Hacer predicciones con los mejores modelos encontrados
y_ridge_pred = best_ridge_model.predict(test_df_scaled) 

# Mostrar las predicciones
print("Predicciones con el modelo Ridge:", y_ridge_pred)


Predicciones con el modelo Ridge: [13.48288318 75.40662505 42.03491911 20.77239146 16.24484546]
